In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Conv3D,MaxPooling3D,UpSampling3D,concatenate,Conv3DTranspose,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, Concatenate
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image 
from read_write_mrc import read_mrc, write_mrc
import skimage 
import torch 
import torchvision as transforms 
from PIL import Image 
import os 

In [2]:
def normalisation(img):
    norm_img = (img-img.min())/(img.max()-img.min())
    return norm_img

In [3]:
def a_hist_eq(img):
    eq_img = skimage.exposure.equalize_adapthist(img)   
    return eq_img

def hist_eq(img):
    eq_img = skimage.exposure.equalize_hist(img)
    return eq_img  

In [4]:
def padding(img):
    
    if (img.shape[1]%2 != 0):
        pad_img = np.pad(img, pad_width= (  (int((704-img.shape[0])/2),int((704-img.shape[0])/2+1))   ,  ( int((704-img.shape[1])/2), int((704-img.shape[1])/2+1)),  ( int((704-img.shape[2])/2), int((704-img.shape[2])/2+1)  ))) 
    
    if (img.shape[1]%2 ==0):
        pad_img = np.pad(img, pad_width= (  (int((704-img.shape[0])/2),int((704-img.shape[0])/2+1))   ,  ( int((704-img.shape[1])/2), int((704-img.shape[1])/2)),  ( int((704-img.shape[2])/2), int((704-img.shape[2])/2+1)  )))
    
    return pad_img  

In [5]:
def slice_mrc(img,file_name,target_location):
    for i in range(len(img.shape)):
        for j in range(img.shape[i]):
            if 32*j<(img.shape[i]-63): 
                if (i ==0):
                    im = img[32*j:32*j+64]
                    # slices.append(im)
                    write_mrc(target_location+'/'+(file_name[0:-4]+'_sliced_'+str(i)+str(j)+'.mrc'), im)
                if (i ==1):
                    im = img[:,32*j:32*j+64,:]
                    im = np.rot90(im, axes=(1,0))
                    # slices.append(im)
                    write_mrc(target_location+'/'+(file_name[0:-4]+'_sliced_'+str(i)+str(j)+'.mrc'), im)
                if (i ==2):
                    im = img[:,:,32*j:32*j+64]
                    im = np.rot90(im , axes = (2,0))
                    # slices.append(im)
                    write_mrc(target_location+'/'+(file_name[0:-4]+'_sliced_'+str(i)+str(j)+'.mrc'), im)

In [6]:
def mask_crop(image, labels):
    mask = np.where(labels != 0, np.full(labels.shape, 255), np.zeros(shape = labels.shape))
    masked = np.where(mask == 255, img, np.zeros(shape = img.shape))
    return (masked,labels)

In [7]:
import os

img_directory = 'demo_data/mrc'
mask_directory = 'demo_data/masks'

img_name_list = os.listdir(img_directory)
mask_name_list =  os.listdir(mask_directory)


img_name_list.sort()
mask_name_list.sort()


for img_name,mask_name in zip(img_name_list, mask_name_list):
    assert str(img_name[0:-4]+str('_labels.mrc')) == str(mask_name), "Mask doesn't match to image"
    
    
    # print(img_name[0:-4])
    # print(mask_name[0:-4])
    
    # print(type(mrc_name))
    
    img_path = os.path.join(img_directory, img_name )
    mask_path = os.path.join(mask_directory,mask_name)
    
    
    # print(img_path)
    # print(mask_path)
    
    img = read_mrc(img_path)
    mask = read_mrc(mask_path)
    
    
    assert img.shape == mask.shape, "Mask shape does not match to image shape."
    
    img = normalisation(img)
    
    img = a_hist_eq(img)
    
    img = padding(img)
    mask = padding(mask)
    
    assert img.shape == (704,704,704), "Something error after padding"
    assert mask.shape == (704,704,704), "Something error after padding"
    
    img,mask = mask_crop(img,mask)

#     # print(mrc_name[0:-4])
#     # print(mask_name[0:-4])
    slice_mrc(img.astype('float32'), img_name, 'slices/images')
    slice_mrc(mask.astype('float32'), mask_name, 'slices/masks')

In [8]:
img.shape

(704, 704, 704)